In [1]:
using Pkg, Revise
Pkg.activate(joinpath(@__DIR__, "../.."))
using EasyABM

  Activating project at `~/Documents/EasyABM`
[ Info: Precompiling EasyABM [2ab8e4d3-e1e9-4248-86a3-8419d2d242e9]


WebIO._IJuliaInit()

In [2]:
Pkg.add("Revise")

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
   Installed CodeTracking ───── v1.3.5
   Installed JuliaInterpreter ─ v0.9.26
   Installed Revise ─────────── v3.5.6
    Updating `~/.julia/environments/v1.9/Project.toml`
  [295af30f] + Revise v3.5.6
    Updating `~/.julia/environments/v1.9/Manifest.toml`
  [da1fd8a2] + CodeTracking v1.3.5
  [aa1ae85d] + JuliaInterpreter v0.9.26
  [6f1432cf] + LoweredCodeUtils v2.3.0
  [295af30f] + Revise v3.5.6
Precompiling project...
  ✓ CodeTracking
  ✓ EasyABM
  ✓ JuliaInterpreter
  ✓ LoweredCodeUtils
  ✓ Revise
  5 dependencies successfully precompiled in 45 seconds. 295 already precompiled.


In [2]:
using BenchmarkTools

In [3]:
@enum agentsfeeling happy sad

In [48]:
typeof(1)

Int64

In [27]:
function return_vals(numagents, griddims, min_to_be_happy, radius)
        agents = grid_2d_agents(numagents, pos = Vect(1,1), space_type=NPeriodic, color = cl"green", mood = happy, keeps_record_of=Set([:pos, :mood]))
        model = create_2d_model(agents, agents_type = Static, space_type=NPeriodic, size = griddims, min_alike = min_to_be_happy)
        
        function initialiser!(model)
            min_alike = model.parameters.min_alike::Int
            for agent in model.agents
                agent.color::Col = [cl"red", cl"green"][rand(1:2)]
                x,y = random_empty_patch(model)::Tuple{Int, Int}
                agent.pos = Vect(x,y)
                 agent.mood = sad
            end    
        end


        function step_rule!(model)
            min_alike = model.parameters.min_alike::Int
            for agent in model.agents
                count_alike = 0
                for nbr in neighbors_moore(agent, model,1)
                    if agent.color::Col == nbr.color::Col
                        count_alike += 1
                    end
                end
                if count_alike ≥ min_alike
                    agent.mood = happy
                else
                    agent.mood = sad
                    x,y = random_empty_patch(model)::Tuple{Int, Int}
                    agent.pos = Vect(x,y)
                end
            end
            return
        end
    return model, initialiser!, step_rule!
end

return_vals (generic function with 1 method)

In [5]:
const agents = grid_2d_agents(1000, pos = Vect(1,1), color = cl"green", mood = happy, keeps_record_of=Set([:pos, :mood]))
const model = create_2d_model(agents, agents_type = Static, space_type=NPeriodic, size = (40,40), min_alike = 3)

EasyABM SpaceModel2D{Static, Int64, NPeriodic}: In a Static model number of agents is fixed.


In [16]:
@code_warntype grid_2d_agents(1000, pos = Vect(1,1), color = cl"green", mood = happy, keeps_record_of=Set([:pos, :mood]))

MethodInstance for Core.kwcall(::NamedTuple{(:pos, :color, :mood, :keeps_record_of), Tuple{Vect{2, Int64}, Col, agentsfeeling, Set{Symbol}}}, ::typeof(grid_2d_agents), ::Int64)
  from kwcall(::Any, ::typeof(grid_2d_agents), n::Int64) @ EasyABM ~/Documents/EasyABM/src/agents/agent2dgrid.jl:43
Arguments
  _::Core.Const(Core.kwcall)
  @_2::NamedTuple{(:pos, :color, :mood, :keeps_record_of), Tuple{Vect{2, Int64}, Col, agentsfeeling, Set{Symbol}}}
  @_3::Core.Const(EasyABM.grid_2d_agents)
  n::Int64
Locals
  pos::Union{}
  space_type::Union{}
  agent_type::Union{}
  kwargs...::Base.Pairs{Symbol, Any, Tuple{Symbol, Symbol, Symbol}, NamedTuple{(:color, :mood, :keeps_record_of), Tuple{Col, agentsfeeling, Set{Symbol}}}}
  @_9::Union{Vect{2, Int64}, Type{Periodic}, Type{Static}}
Body::Vector{EasyABM.Agent2D{Int64, Periodic, Static}}
1 ──       Core.NewvarNode(:(pos))
│          Core.NewvarNode(:(space_type))
│          Core.NewvarNode(:(agent_type))
│          Core.NewvarNode(:(kwargs...))
│    

In [12]:
@code_warntype run_model!(model, steps=20, step_rule = step_rule!)

MethodInstance for Core.kwcall(::NamedTuple{(:steps, :step_rule), Tuple{Int64, typeof(step_rule!)}}, ::typeof(run_model!), ::EasyABM.SpaceModel2D{Static, Int64, NPeriodic})
  from kwcall(::Any, ::typeof(run_model!), model::EasyABM.SpaceModel2D) @ EasyABM ~/Documents/EasyABM/src/2d_model/mainfunctions.jl:155
Arguments
  _::Core.Const(Core.kwcall)
  @_2::NamedTuple{(:steps, :step_rule), Tuple{Int64, typeof(step_rule!)}}
  @_3::Core.Const(EasyABM.run_model!)
  model::EasyABM.SpaceModel2D{Static, Int64, NPeriodic}
Locals
  steps::Union{}
  step_rule::Union{}
  @_7::Union{typeof(step_rule!), Int64}
Body::Nothing
1 ──       Core.NewvarNode(:(steps))
│          Core.NewvarNode(:(step_rule))
│          Core.NewvarNode(:(@_7))
│    %4  = Core.isdefined(@_2, :steps)::Core.Const(true)
└───       goto #3 if not %4
2 ──       (@_7 = Core.getfield(@_2, :steps))
└───       goto #4
3 ──       Core.Const(:(@_7 = 1))
4 ┄─ %9  = @_7::Int64
│    %10 = Core.isdefined(@_2, :step_rule)::Core.Const(true)
└───

In [28]:

function bench_easyabm_model(numagents, griddims, min_to_be_happy, radius)
#@enum agentsfeeling happy sad
    model, initialiser!, step_rule! = return_vals(numagents, griddims, min_to_be_happy, radius)
    init_model!(model, initialiser = initialiser!)
    run_model!(model, steps=20, step_rule = step_rule!)
end

bench_easyabm_model (generic function with 1 method)

In [30]:
@time getfield(model.agents[1], :pos)

  0.000003 seconds


(10, 30)


In [32]:
@time getfield(model.agents[1], :last_grid_loc)

  0.000003 seconds


(1, 1)

In [28]:
@time model.agents[1].pos

  0.000003 seconds


(10, 30)


In [40]:
@code_warntype getfield(model.agents[1],:pos)

getfield(...) @ Core none:0
  failed to infer


In [31]:
@benchmark bench_easyabm_model(1000, (40, 40), 3, 1)

BenchmarkTools.Trial: 94 samples with 1 evaluation.
 Range (min … max):  44.629 ms … 71.867 ms  ┊ GC (min … max):  9.73% … 19.89%
 Time  (median):     53.149 ms              ┊ GC (median):    22.96%
 Time  (mean ± σ):   53.414 ms ±  4.746 ms  ┊ GC (mean ± σ):  20.19% ±  5.34%

              ▆▁ ▁ ▃  ▁█▃▁▄                                    
  ▄▆▄▁▄▄▇▄▁▁▁▇██▆█▁█▆▇█████▆▇▇▆▄▄▁▄▁▇▇▁▄▁▁▁▁▁▄▆▁▄▁▁▁▁▄▁▁▁▁▁▁▄ ▁
  44.6 ms         Histogram: frequency by time        68.2 ms <

 Memory estimate: 67.57 MiB, allocs estimate: 121826.

In [32]:
@benchmark bench_easyabm_model(5000, (80, 80), 3, 1) evals=1

BenchmarkTools.Trial: 9 samples with 1 evaluation.
 Range (min … max):  516.232 ms … 632.301 ms  ┊ GC (min … max): 14.85% … 13.02%
 Time  (median):     566.141 ms               ┊ GC (median):    13.84%
 Time  (mean ± σ):   577.140 ms ±  36.246 ms  ┊ GC (mean ± σ):  13.44% ±  1.09%

  █                    █ █ ███                    █     █     █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁█▁███▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁█ ▁
  516 ms           Histogram: frequency by time          632 ms <

 Memory estimate: 907.10 MiB, allocs estimate: 552220.

In [33]:
@code_warntype bench_easyabm_model(1000, (40, 40), 3, 1)

MethodInstance for bench_easyabm_model(::Int64, ::Tuple{Int64, Int64}, ::Int64, ::Int64)
  from bench_easyabm_model(numagents, griddims, min_to_be_happy, radius) @ Main In[28]:2
Arguments
  #self#::Core.Const(bench_easyabm_model)
  numagents::Int64
  griddims::Tuple{Int64, Int64}
  min_to_be_happy::Int64
  radius::Int64
Locals
  @_6::Int64
  step_rule!::var"#step_rule!#10"
  initialiser!::var"#initialiser!#9"
  model::EasyABM.SpaceModel2D
Body::Nothing
1 ─ %1  = Main.return_vals(numagents, griddims, min_to_be_happy, radius)::Tuple{EasyABM.SpaceModel2D, var"#initialiser!#9", var"#step_rule!#10"}
│   %2  = Base.indexed_iterate(%1, 1)::Core.PartialStruct(Tuple{EasyABM.SpaceModel2D, Int64}, Any[EasyABM.SpaceModel2D, Core.Const(2)])
│         (model = Core.getfield(%2, 1))
│         (@_6 = Core.getfield(%2, 2))
│   %5  = Base.indexed_iterate(%1, 2, @_6::Core.Const(2))::Core.Const((var"#initialiser!#9"(), 3))
│         (initialiser! = Core.getfield(%5, 1))
│         (@_6 = Core.getfield(%5, 

In [14]:
using Base.Threads

In [15]:
function loop1()
    @threads for i in 1:1000
        x=rand()
        if x<0.05
            println("num ", i, " threadId ", Threads.threadid() )
            return 
        end
    end
end

loop1 (generic function with 1 method)

In [16]:
loop1()

num 77 threadId 3
num 257 threadId 2
num 537 threadId 5
num 170 threadId 4
num 844 threadId 6
num 678 threadId 8
num 380 threadId 1
num 957 threadId 7


In [61]:
rand()

0.11153748856225099

In [17]:
nthreads()

8

In [18]:
vals = [:a, :b,:c, :d, :e, :f, :g, :h,:i,:j]



10-element Vector{Symbol}:
 :a
 :b
 :c
 :d
 :e
 :f
 :g
 :h
 :i
 :j

In [20]:
for (i, x) in enumerate(vals)
    println(i," ", x)
end

1 a
2 b
3 c
4 d
5 e
6 f
7 g
8 h
9 i
10 j


In [23]:
mat = [i+j for i in 1:3, j in 1:3]

3×3 Matrix{Int64}:
 2  3  4
 3  4  5
 4  5  6

In [25]:
filter(val->val%2==0, mat)

5-element Vector{Int64}:
 2
 4
 4
 4
 6